## Parameters

In [ ]:
input_dir = "/research/sharedresources/cbi/common/Anna/test/input"
segm_dir = "/research/sharedresources/cbi/common/Anna/test/Analysis/cell_segmentation"
output_dir = "/research/sharedresources/cbi/common/Anna/test/Analysis/parameter_sweep"
output_code_dir = "outputs_parameter_sweep"


cell_stats_dir = 'quantification_cells'
puncta_stats_dir = 'quantification_puncta'
puncta_segm_dir = 'puncta_segmentation_with_raw'


minsize_um = 0.2  
maxsize_um = 2  
num_sigma = 5  

overlap = 1  
threshold_detection = 0.003 
threshold_segmentation = 0.001  

threshold_background = 3

max_threads = 20   
segmentation_mode = 0

In [ ]:
#################################
#  Don't modify the code below  #
#################################
import os
code_dir = os.getcwd()
os.makedirs(os.path.join(code_dir, output_code_dir), exist_ok=True)

## Loop over parameter values

Please change the list of values in the loop according to your needs.

Also, you can change the parameters to loop over, and add new parameters/values to the `name` variable that is used to name the output folders

In [ ]:
for threshold_detection in [0.003, 0.002]:
    for threshold_segmentation in [0.001]:
        for threshold_background in [3]:
            name = rf"thr_detection={threshold_detection}_thr_segm={threshold_segmentation}"\
                   rf"_thr_bg={threshold_background}" 
            csd = rf"{cell_stats_dir}/{name}"
            psd = rf"{puncta_stats_dir}/{name}"
            psegd = rf"{puncta_segm_dir}/{name}" 
            print(name)
            command = rf'source activate nup98_puncta; cd {code_dir}; papermill analyze_peaks_per_cell.ipynb ' \
                  rf' {output_code_dir}/analyze_peaks_per_cell_{name}.ipynb -p max_threads {max_threads} ' \
                  rf' -p input_dir {input_dir} -p output_dir {output_dir} -p segm_dir {segm_dir} ' \
                  rf' -p minsize_um {minsize_um} -p maxsize_um {maxsize_um} -p num_sigma {num_sigma}' \
                  rf' -p overlap {overlap} -p threshold_background {threshold_background}' \
                  rf' -p threshold_detection {threshold_detection} ' \
                  rf' -p segmentation_mode {segmentation_mode} ' \
                  rf' -p threshold_segmentation {threshold_segmentation}' \
                  rf' -p cell_stats_dir {csd} -p puncta_stats_dir {psd} -p puncta_segm_dir {psegd} ' \
                  rf' -k python3 ' 
            os.system(command)

## Combine all statistics

In [ ]:
#################################
#  Don't modify the code below  #
#################################

import pandas as pd
for stat_dir in [cell_stats_dir, puncta_stats_dir]:
    stat_combined = pd.DataFrame()
    files = os.listdir(os.path.join(output_dir, stat_dir))
    for fn in files:
        if fn.endswith('csv'):
            stat = pd.read_csv(os.path.join(output_dir, stat_dir, fn))
            stat_combined = pd.concat([stat_combined, stat], ignore_index=True)
    stat_combined.to_csv(os.path.join(output_dir, stat_dir + '_combined.csv'), index=False)
    